In [1]:
# re
import re

# POS
import spacy

# nltk for wordnet
import nltk
from nltk.corpus import wordnet as wn

In [2]:
nlp = spacy.load('en_core_web_sm', disable=['ner,parser'])
nlp.remove_pipe('ner')
nlp.remove_pipe('parser')

('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x13e5ff8e0>)

In [3]:
'''
Reads in a file containing text samples, that has each sample separated by a carriage return.

Args:
    filename: location of sample file

Return:
    tagged_samples: a list of spacy `nlp` docs that contains information about each word in the sample
'''
def read_and_tag(filename):    
    
    tagged_samples = []
    
    with open(filename, encoding="utf-8") as file:
        # samples are separated by line breaks
        for sample in file:            
            doc=nlp(sample)
            tagged_samples.append(doc)
        
    return tagged_samples

In [4]:
# Function from http://www.nltk.org/howto/wordnet.html to get *all* of a synset's hyponym/hypernyms
hyper = lambda s: s.hypernyms()

In [5]:
'''
Consult wordnet for the situation of a verb or noun with respect to its station 
In the hypernym hierarchy. 
Based on current SOA, it is acceptable to simply grab the top-level (.01) synset.

Args:
    tagged_sample: a spacy doc

Return:
    specificity: a value conveying the "specificity" of the input, via Nelson (2020)
'''

def specificity(tagged_sample):
    hyper_sum = 0
    noun_and_verb_count = 0
    for word in tagged_sample:
        if word.pos_ == "NOUN" or word.pos_ == "VERB":
            # if it's a verb, get the most common verb hypernym chain
            # else, get the most common noun hypernym chain
            pos = word.pos_
            tag = "n" if pos.startswith("N") else "v"
            wn_lookup = word.lemma_ + "." + tag + ".01"
            try:
                hyper_sum += len(list(wn.synset(wn_lookup).closure(hyper)))
            except:
                # on off chance we have a mistag, don't break down the system
                continue
            noun_and_verb_count +=1
    
    return hyper_sum / noun_and_verb_count

In [6]:
def pos_counts(tagged_sample):
    verb_count = 0
    adj_count = 0
    adv_count = 0
    noun_count = 0
    
    for word in tagged_sample:
        if word.pos_ == "NOUN":
            noun_count +=1
        elif word.pos_ == "VERB":
            verb_count +=1
        elif word.pos_ == "ADJ":
            adj_count +=1
        elif word.pos_ == "ADV":
            adv_count +=1
    
    return str(adj_count) + "," + str(adv_count) + "," + str(noun_count) + "," + str(verb_count)

In [7]:
detail_samples = read_and_tag("detail_samples.txt")

In [8]:
not_detail_samples = read_and_tag("not_detail_samples.txt")

In [9]:
detail_attr = []
for sample in detail_samples:
    detail_attr.append(str(specificity(sample)) + "," + pos_counts(sample))

/opt/anaconda3/envs/anlp/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: UserWarning: Discarded redundant search for Synset('substance.n.01') at depth 4
  for synset in acyclic_breadth_first(self, rel, depth):
/opt/anaconda3/envs/anlp/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: UserWarning: Discarded redundant search for Synset('entity.n.01') at depth 7
  for synset in acyclic_breadth_first(self, rel, depth):
/opt/anaconda3/envs/anlp/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: UserWarning: Discarded redundant search for Synset('physical_entity.n.01') at depth 7
  for synset in acyclic_breadth_first(self, rel, depth):
/opt/anaconda3/envs/anlp/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: UserWarning: Discarded redundant search for Synset('physical_entity.n.01') at depth 9
  for synset in acyclic_breadth_first(self, rel, depth):
/opt/anaconda3/envs/anlp/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: Use

/opt/anaconda3/envs/anlp/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: UserWarning: Discarded redundant search for Synset('biological_group.n.01') at depth 3
  for synset in acyclic_breadth_first(self, rel, depth):
/opt/anaconda3/envs/anlp/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: UserWarning: Discarded redundant search for Synset('animal.n.01') at depth 7
  for synset in acyclic_breadth_first(self, rel, depth):
/opt/anaconda3/envs/anlp/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: UserWarning: Discarded redundant search for Synset('entity.n.01') at depth 4
  for synset in acyclic_breadth_first(self, rel, depth):
/opt/anaconda3/envs/anlp/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: UserWarning: Discarded redundant search for Synset('structure.n.01') at depth 6
  for synset in acyclic_breadth_first(self, rel, depth):
/opt/anaconda3/envs/anlp/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: UserWarning

In [10]:
not_detail_attr = []
for sample in not_detail_samples:
    not_detail_attr.append(str(specificity(sample)) + "," + pos_counts(sample))

/opt/anaconda3/envs/anlp/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: UserWarning: Discarded redundant search for Synset('psychological_feature.n.01') at depth 7
  for synset in acyclic_breadth_first(self, rel, depth):
/opt/anaconda3/envs/anlp/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: UserWarning: Discarded redundant search for Synset('event.n.01') at depth 6
  for synset in acyclic_breadth_first(self, rel, depth):
/opt/anaconda3/envs/anlp/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: UserWarning: Discarded redundant search for Synset('psychological_feature.n.01') at depth 9
  for synset in acyclic_breadth_first(self, rel, depth):
/opt/anaconda3/envs/anlp/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: UserWarning: Discarded redundant search for Synset('animal.n.01') at depth 12
  for synset in acyclic_breadth_first(self, rel, depth):
/opt/anaconda3/envs/anlp/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py

In [18]:
'''
Helper method for writing an inputted data string (comma delimited, already) to a file.
Args:
    text: text of the book (str)
    filename: filename, should have directory prefixed already (str)

'''
def write_sample_data_to_file(text, filename):
        
    if text is None:
        return
    f = open(filename, "w")
    for val in text:
        f.write(val)
        f.write("\n")
    f.close()

In [19]:
write_sample_data_to_file(detail_attr, "detail_sample_data.csv")

In [20]:
write_sample_data_to_file(not_detail_attr, "not_detail_sample_data.csv")